# Register model and deploy locally

See: https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/deployment/deploy-to-local/register-model-deploy-local.ipynb

- Register model
- Deploy the image as a web service in a local Docker container.
- Quickly test changes to your entry script by reloading the local service.
- Optionally, you can also make changes to model, conda or extra_docker_file_steps and update local service

## Assumptions

- Data is already preprocessed in `data/`

# Setup Workspace and Prereqs

In [1]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.50.0


In [2]:
from azureml.core.workspace import Workspace

# the config file is one directory up
# NOTE: this is an interactive authorization!
ws = Workspace.from_config('..')
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

Nanostics_ML_Workspace
SummerProjects2023
canadacentral
1b1ae7cf-df24-428b-8bb9-e4dd07869ac9


# Train Model

the code is ripped from `../train.ipynb`, just without the MLFlow stuff

In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd 

def split_and_scale(df):
    y = df["diagnosis_01"]
    X = df.drop(["diagnosis_01"],axis=1)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    return X_train, X_test, y_train, y_test, X.columns

In [4]:
from sklearn.svm import SVC

def train_svc(df):
    X_train, X_test, y_train, y_test, train_cols = split_and_scale(df)
    svc = SVC()
    svc.fit(X_train, y_train)
    return svc, X_test, y_test, train_cols

In [5]:
import pandas as pd 


data = '../data/cleaned-wisconsin-lof.parquet'
df = pd.read_parquet(data)

svc, X_test, y_test, train_cols = train_svc(df)

In [6]:
# df_test = pd.DataFrame(X_test, columns=df.columns.drop('diagnosis_01'))

# d = df_test.iloc[0].to_dict()
# # # pd.DataFrame(d, index=[0])
# # svc.predict(pd.DataFrame(d, index=[0]))
X_test[0]

array([-0.74770417,  0.17298137, -0.31968485, -0.70288103, -0.64621089,
       -0.27678574, -0.15264977, -1.02292851, -0.58514392, -1.01275854,
       -0.77209937, -1.37062653, -0.71853825, -0.545025  , -0.73391292,
       -0.22507314, -0.67222361, -0.44056926, -0.51013498, -0.52695596,
       -0.6780146 , -0.38722193, -0.45080674, -0.46281914,  0.71062877,
       -0.33474174])

In [7]:
import joblib

joblib.dump(svc, 'model.pkl')

['model.pkl']

## Register Model

In [8]:
from azureml.core.model import Model

model = Model.register(model_path="model.pkl",
                       model_name="wisconsin-BCa-model",
                       tags={'area': "cancer", 'type': "svc"},
                       description="SVC model to predict Breast Cancer",
                       workspace=ws)

Registering model wisconsin-BCa-model


## Create Environment + Inference Config

straight copy-paste

In [9]:
import sklearn

from azureml.core.environment import Environment

environment = Environment("LocalDeploy")
environment.python.conda_dependencies.add_pip_package("inference-schema[numpy-support]")
environment.python.conda_dependencies.add_pip_package("joblib")
environment.python.conda_dependencies.add_pip_package("scikit-learn=={}".format(sklearn.__version__))

In [10]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(entry_script="score.py",
                                   environment=environment)

## Deploy Model as a Docker Service

This is the good stuff!!!!

In [11]:
from azureml.core.webservice import LocalWebservice

# This is optional, if not provided Docker will choose a random unused port.
deployment_config = LocalWebservice.deploy_configuration(port=6789)

local_service = Model.deploy(ws, "test", [model], inference_config, deployment_config)

local_service.wait_for_deployment()

/var/folders/t3/6z39454j7vd3kskzz39tmm6m0000gn/T/ipykernel_77864/1718659047.py:6: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  local_service = Model.deploy(ws, "test", [model], inference_config, deployment_config)


Generating Docker build context.
2023/05/16 16:33:01 Downloading source code...
2023/05/16 16:33:07 Finished downloading source code
2023/05/16 16:33:07 Creating Docker network: acb_default_network, driver: 'bridge'
2023/05/16 16:33:08 Successfully set up Docker network: acb_default_network
2023/05/16 16:33:08 Setting up Docker configuration...
2023/05/16 16:33:08 Successfully set up Docker configuration
2023/05/16 16:33:08 Logging in to registry: nanosticsmlregistry.azurecr.io
2023/05/16 16:33:09 Successfully logged into nanosticsmlregistry.azurecr.io
2023/05/16 16:33:09 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2023/05/16 16:33:09 Scanning for dependencies...
2023/05/16 16:33:09 Successfully scanned dependencies
2023/05/16 16:33:09 Launching container with name: acb_step_0
Sending build context to Docker daemon  71.68kB
Step 1/21 : FROM mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20230324.v1@sha256:baf6420f6eac8aa7

In [12]:
print('Local service port: {}'.format(local_service.port))


Local service port: 6789


## Test Web Service


In [13]:
print(local_service.get_logs())


/bin/bash: /azureml-envs/azureml_c804bc43f202681eba163665491706a1/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_c804bc43f202681eba163665491706a1/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_c804bc43f202681eba163665491706a1/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2023-05-16T16:51:22,270989801+00:00 - rsyslog/run 
2023-05-16T16:51:22,272249384+00:00 - nginx/run 
bash: /azureml-envs/azureml_c804bc43f202681eba163665491706a1/lib/libtinfo.so.6: no version information available (required by bash)
2023-05-16T16:51:22,314987259+00:00 - gunicorn/run 
2023-05-16T16:51:22,347727884+00:00 | gunicorn/run | 
2023-05-16T16:51:22,382501384+00:00 | gunicorn/run | ###############################################
2023-05-16T16:51:22,413155634+00:00 | gunicorn/run | AzureML Container Runtime Information
2023-05-16T16:51:22,441842509+00:00 | gunico

In [16]:
import json

sample_input = json.dumps({
    'data': X_test[0:10].tolist()
})

local_service.run(sample_input)

[0, 1, 1, 0, 0, 0, 1, 0, 1, 0]

## Delete Service

In [1]:
local_service.delete()

NameError: name 'local_service' is not defined